Предположим, что в коллекции 'Account' одноименного класса лежит список документов вида:

In [55]:
input_json = {
    'number': '7800000000000',
    'name': 'Пользователь №',
    'sessions': [
        {
            'created_at': ISODate('2016-01-01T00:00:00'),
            'session_id': '6QBnQhFGgDgC2FDfGwbgEaLbPMMBofPFVrVh9Pn2quooAcgxZc',
            'actions': [
                {
                    'type': 'read',
                    'created_at': ISODate('2016-01-01T01:20:01'),
                },
                {
                    'type': 'read',
                    'created_at': ISODate('2016-01-01T01:21:13'),
                },
                {
                    'type': 'create',
                    'created_at': ISODate('2016-01-01T01:33:59'),
                }
            ],
        }
    ]
}

Необходимо написать агрегационный запрос, который по каждому пользователю выведет последнее действие и общее количество для каждого из типов 'actions'. Итоговые данные должны представлять собой список документов вида:

In [56]:
output_json = {
    'number': '7800000000000',
    'actions': [
        {
            'type': 'create',
            'last': 'created_at': ISODate('2016-01-01T01:33:59'),
            'count': 12,
        },
        {
            'type': 'read',
            'last': 'created_at': ISODate('2016-01-01T01:21:13'),
            'count': 12,
        },
        {
            'type': 'update',
            'last': null,
            'count': 0,
        },
        {
            'type': 'delete',
            'last': null,
            'count': 0,
        },
    ]
}

SyntaxError: invalid syntax (<ipython-input-56-917c7482d833>, line 6)

Для корректного выполнения данной задачи, примем некоторые допущения:

- Допустим, что нам необходимо сохранить исходный формат ISODate, не читая его содержимого, для этого, введем псевдо-функцию (и для удобства отображения результата):

In [57]:
def ISODate(str): # да простит меня pep-8..
    return 'ISODate(' + str + ')'

- Допустим, что в условии задачи допущена ошибка:

In [58]:
{
'last': 'created_at': ISODate('2016-01-01T01:33:59'),
}

SyntaxError: invalid syntax (<ipython-input-58-12411ed94b84>, line 2)

В этом случае, данная строка может принять вид:

In [59]:
{
'last': ISODate('2016-01-01T01:33:59'),
}

{'last': 'ISODate(2016-01-01T01:33:59)'}

или

In [60]:
{
'last': 
    {'created_at': ISODate('2016-01-01T01:33:59'),}
}

{'last': {'created_at': 'ISODate(2016-01-01T01:33:59)'}}

- Используем первый вариант, поскольку он кажется более логичным с точки зрения избыточности данных.

In [61]:
class User(object):

    def __init__(self):
        self.number = ''
        self.actions = [
            {'type': 'create', 'last': None, 'count': 0},
            {'type': 'read', 'last': None, 'count': 0},
            {'type': 'update', 'last': None, 'count': 0},
            {'type': 'delete', 'last': None, 'count': 0},
        ]

    def control(self, input_json):
        self.number = input_json['number']
        for session in input_json['sessions']:
            for action in session['actions']:
                if action['type'] == 'create':
                    self.actions[0]['count'] += 1
                    self.actions[0]['last'] = action['created_at']
                elif action['type'] == 'read':
                    self.actions[1]['count'] += 1
                    self.actions[1]['last'] = action['created_at']
                elif action['type'] == 'update':
                    self.actions[2]['count'] += 1
                    self.actions[2]['last'] = action['created_at']
                elif action['type'] == 'delete':
                    self.actions[3]['count'] += 1
                    self.actions[3]['last'] = action['created_at']

    def viewer(self):
        return {'number': self.number, 'actions': self.actions}

Инициализируем новый объект класса User.

In [62]:
my_user = User()

Проходимся по исходному массиву данных, собирая все действия пользователя и его условный идентификатор 'number' в виде аттрибутов данного объекта.

In [63]:
my_user.control(input_json)

Выводим содержимое на экран (можно все закинуть в одну функцию, но у нас же вроде MVC  :)

In [64]:
print(my_user.viewer())

{'number': '7800000000000', 'actions': [{'type': 'create', 'last': 'ISODate(2016-01-01T01:33:59)', 'count': 1}, {'type': 'read', 'last': 'ISODate(2016-01-01T01:21:13)', 'count': 2}, {'type': 'update', 'last': None, 'count': 0}, {'type': 'delete', 'last': None, 'count': 0}]}
